In [92]:
# Импорт библиотек
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_raw = root / 'data' / 'raw_data'
data_out = root / 'data' / 'processed_data'

print("Пути инициализированы:")
print(f"Raw data: {data_raw}")
print(f"Processed data: {data_out}")

Пути инициализированы:
Raw data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/raw_data
Processed data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data


In [94]:
# Загрузка данных
print("\nЗагружаем сырые данные...")
sessions_raw = pd.read_pickle(data_raw / 'ga_sessions.pkl')
hits_raw = pd.read_pickle(data_raw / 'ga_hits.pkl')
print("\nДанные успешно загружены!")


Загружаем сырые данные...

Данные успешно загружены!


In [96]:
def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")
    
    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return
    
    # Вывод типа данных
    print(f"Тип данных: {df[col].dtype}")
    
    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")
    
    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())
        
    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-10 значений:")
        print(df[col].value_counts(dropna=False).head(20))


generate_summary(sessions_raw, "Сырые данные сессий")
for col in sessions_raw.columns:
    analyze_column(sessions_raw, col, max_display=100)     


================================================== СЫРЫЕ ДАННЫЕ СЕССИЙ ==================================================
Общее количество записей: 1,860,042
Количество признаков: 18

Типы данных:
        count
object     17
int64       1

Пропущенные значения:
                 count     %
device_model   1843704 99.12
utm_keyword    1082061 58.17
device_os      1070138 57.53
utm_adcontent   335615 18.04
utm_campaign    219603 11.81
device_brand    118678  6.38
utm_source          97  0.01

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867...,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.164064...,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: object
Пропуски: 0 (0.0%)
Уникальных значений: 1860042

Слишком много значений (> 100). Примеры:
['8317592486913541296.1626375343.1626375343'
 '1046779193471368056.1629584251.1629584251'
 '972596947823925584.1621924176.1621924176'
 '399644702783159829.1640087063.1640087063'
 '4560533380461397178.1623079611.1623079611'
 '615203545085975675.1634278526.1634278526'
 '5171683235009347213.1637039735.1637039735'
 '5407122024939374752.1639172245.1639172245'
 '648841153410219293.1621867807.1621867807'
 '8103675271307569516.1634183533.1634183533']

Топ-10 значений:
session_id
9055434745589932991.1637753792.1637753792    1
6294640354816492625.1638728787.1638728809    1
6294832296889291185.1623245519.1623245519    1
6294832296889291185.1623161065.1623161065    1
6294832296889291185.1623156608.1623156608    1
6294832296889291185.1623069105.1623069105    1
6294825034110946181.1634421637.163442

In [167]:
# Копируем данные
sessions = sessions_raw.copy()

# Объединяем 'visit_date' и 'visit_time' в 'visit_datetime'
sessions['visit_datetime'] = pd.to_datetime(
    sessions['visit_date'].astype(str) + ' ' + sessions['visit_time'].astype(str),
    errors='coerce'
)

# Удаляем старые столбцы 'visit_date' и 'visit_time'
sessions.drop(columns=['visit_date', 'visit_time'], inplace=True)

# Вставляем 'visit_datetime' после 'client_id'
sessions.insert(sessions.columns.get_loc('client_id') + 1, 'visit_datetime', sessions.pop('visit_datetime'))

# Извлекаем дополнительные данные из 'visit_datetime'
sessions['visit_hour'] = sessions['visit_datetime'].dt.hour
sessions['visit_weekday'] = sessions['visit_datetime'].dt.weekday
sessions['is_weekend'] = sessions['visit_weekday'] >= 5

# Добавляем новые столбцы после 'visit_datetime'
for col in ['is_weekend', 'visit_weekday', 'visit_hour']:
    sessions.insert(sessions.columns.get_loc('visit_datetime') + 1, col, sessions.pop(col))

# Удаляем ненужный столбец 'device_model'
sessions.drop(columns=['device_model'], inplace=True)

# Выделяем два отдельных признака высоту и ширину экрана
resolution = (
        sessions['device_screen_resolution']
        .str.extract(r'(?P<screen_width>\d+)x(?P<screen_height>\d+)')
    )

sessions['screen_width'] = resolution['screen_width'].astype('Int32')
sessions['screen_height'] = resolution['screen_height'].astype('Int32')

# Удаляем старый столбец 'device_screen_resolution'
sessions.drop(columns=['device_screen_resolution'], inplace=True)

# Приводим столбцы к правильным типам и заменяем значения
sessions = sessions.convert_dtypes()

# Заменяем значения в нескольких столбцах на pd.NA
replace_dict = {
    'utm_medium': ['(none)', 'nan', '(not set)', 'NaN', 'None', ''],
    'utm_adcontent': ['(none)', 'nan', '(not set)', 'NaN', 'None', ''],
    'utm_keyword': ['(none)', 'nan', '(not set)', 'NaN', 'None', ''],
    'device_os': ['(none)', 'nan', '(not set)', 'NaN', 'None', ''],
    'device_brand': ['(none)', 'nan', '(not set)', 'NaN', 'None', ''],
    'geo_country': ['(none)', 'nan', '(not set)', 'NaN', 'None', ''],
    'geo_city': ['(none)', 'nan', '(not set)', 'NaN', 'None', ''],
    'screen_width':[0],
    'screen_height': [0]
    
}

for col, values in replace_dict.items():
    for value in values:
        sessions[col] = sessions[col].replace(value, pd.NA)
        
# Удаляем строки с пропущенным значением (количество <100 строк)
sessions.dropna(subset=['utm_source'], inplace=True)
sessions.dropna(subset=['screen_width'], inplace=True)
sessions.dropna(subset=['screen_height'], inplace=True)

# Генерация отчета
generate_summary(sessions, "Данные сессий")
for col in sessions.columns:
    analyze_column(sessions, col, max_display=100)



================================================== ДАННЫЕ СЕССИЙ ==================================================
Общее количество записей: 1,859,926
Количество признаков: 20

Типы данных:
                count
string[python]     13
Int32               4
datetime64[ns]      1
boolean             1
Int64               1

Пропущенные значения:
                 count     %
utm_keyword    1082047 58.18
device_os      1070488 57.56
device_brand    384662 20.68
utm_adcontent   335573 18.04
utm_medium      301010 16.18
utm_campaign    219600 11.81
geo_city         78172  4.20
geo_country       1146  0.06

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_datetime,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_browser,geo_country,geo_city,screen_width,screen_height
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,Chrome,Russia,Zlatoust,360,720
1,905544597018549464.1636867...,210838531.1636867288,2021-11-14 08:21:30,8,6,True,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,Samsung Internet,Russia,Moscow,385,854
2,9055446045651783499.164064...,2108385331.1640648523,2021-12-28 02:42:06,2,1,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk,360,720



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1859926

Слишком много значений (> 100). Примеры:
<StringArray>
['2578537644176393116.1629176733.1629176733',
 '1061535537156746165.1622043573.1622043573',
 '3211803307524088639.1634067262.1634067262',
 '3679567860721388356.1639009243.1639009243',
 '2632920696164654149.1621826632.1621826632',
 '9186969991631943038.1634022782.1634022782',
 '6950821382358646841.1621966906.1621966906',
 '1690321107234643485.1628535329.1628535352',
 '5224517032159668411.1624959167.1624959167',
 '7241341388414461210.1640441116.1640441116']
Length: 10, dtype: string

Топ-10 значений:
session_id
9055434745589932991.1637753792.1637753792    1
9055651392319212104.1626688072.1626688072    1
9055447046360770272.1622255345.1622255345    1
9055447192389856083.1622453074.1622453074    1
9055455318486370642.1640843788.1640843788    1
9055461992850812764.1626107740.

In [152]:
generate_summary(hits_raw, "Сырые данные событий")
for col in hits_raw.columns:
    analyze_column(hits_raw, col, max_display=100)


================================================== СЫРЫЕ ДАННЫЕ СОБЫТИЙ ==================================================
Общее количество записей: 15,726,470
Количество признаков: 11

Типы данных:
         count
object       9
float64      1
int64        1

Пропущенные значения:
                count      %
event_value  15726470 100.00
hit_time      9160322  58.25
hit_referer   6274804  39.90
event_label   3760184  23.91

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.164025...,2021-12-23,597864.00,30,event,NaN,sberauto.com/cars?utm_sour...,quiz,quiz_show,NaN,None
1,7750352294969115059.164027...,2021-12-23,597331.00,41,event,NaN,sberauto.com/cars/fiat?cit...,quiz,quiz_show,NaN,None
2,885342191847998240.1640235...,2021-12-23,796252.00,49,event,NaN,sberauto.com/cars/all/volk...,quiz,quiz_show,NaN,None



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: object
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
['8315147834486346153.1634390440.1634390440'
 '1283099933453469139.1624914787.1624914787'
 '8661609806726923122.1630135272.1630135272'
 '6747317795995629216.1627240096.1627240096'
 '6087611596713910728.1622041105.1622041105'
 '5332251536544475337.1623435465.1623435465'
 '8971196306472518719.1637303362.1637303362'
 '5685503203563647978.1640084459.1640084459'
 '8771388041244036552.1635110344.1635110344'
 '6708213128096399477.1640775800.1640775800']

Топ-10 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.1629319807.1629319807    496
686125592720823356.16

In [177]:
# Копируем данные
hits = hits_raw.copy()

# Удаляем ненужный столбец 'event_value'
hits.drop(columns=['event_value', 'hit_type'], inplace=True)

# Приводим столбцы к правильным типам и заменяем значения
hits = hits.convert_dtypes()

generate_summary(hits, "Данные событий")
for col in hits.columns:
    analyze_column(hits, col, max_display=100)

# продолжение следует, где будем объеденять по айди сессий
#в очередной раз и смотреть что можно почистить


================================================== ДАННЫЕ СОБЫТИЙ ==================================================
Общее количество записей: 15,726,470
Количество признаков: 9

Типы данных:
                count
string[python]      6
Int64               2
object              1

Пропущенные значения:
               count     %
hit_time     9160322 58.25
hit_referer  6274804 39.90
event_label  3760184 23.91

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,hit_date,hit_time,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label
0,5639623078712724064.164025...,2021-12-23,597864,30,<NA>,sberauto.com/cars?utm_sour...,quiz,quiz_show,<NA>
1,7750352294969115059.164027...,2021-12-23,597331,41,<NA>,sberauto.com/cars/fiat?cit...,quiz,quiz_show,<NA>
2,885342191847998240.1640235...,2021-12-23,796252,49,<NA>,sberauto.com/cars/all/volk...,quiz,quiz_show,<NA>



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
<StringArray>
['5059330979358694414.1640881164.1640881164',
 '4885032852770059747.1635317221.1635317221',
 '1942738203524766805.1621924950.1621924950',
  '596437059142487859.1628507956.1628507956',
 '6254195231891655015.1624770919.1624770919',
 '3998325876571625493.1632491546.1632491546',
   '24191718466202514.1638869908.1638869908',
 '2662802304068989120.1629466816.1629466816',
 '3925235282777835895.1640860056.1640860056',
 '7629937814464623328.1635513066.1635513066']
Length: 10, dtype: string

Топ-10 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.1